In [1]:
import pandas as pd
import json



df = pd.read_csv("JoSAA 2022 Seat Allocation Tool - For Surya.csv", header=0)
# print(df.head())
seat_types = df["Seat Type"].unique()
print(seat_types)

['OPEN' 'EWS' 'OBC-NCL' 'SC' 'ST' 'OPEN (PwD)' 'EWS (PwD)' 'OBC-NCL (PwD)'
 'SC (PwD)' 'ST (PwD)']


In [ ]:
for seat_type in seat_types:
    df_seat_type = df[df["Seat Type"] == seat_type]
    df_seat_type.to_json(f"public/data/{seat_type}.json", orient="records")

In [42]:
df.columns

Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
       'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
       'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
       'Category Match', '500', 'OBC-NCL'],
      dtype='object')

In [18]:
df["Gender"].unique()

array(['Gender-Neutral', 'Female-only (including Supernumerary)'],
      dtype=object)

In [22]:
df["Exam"].unique()

array(['JEE Advanced', 'JEE Main'], dtype=object)

In [23]:
df["State"].unique()

array(['Odisha', 'Maharashtra', 'Himachal Pradesh', 'New Delhi',
       'Madhya Pradesh', 'West Bengal', 'Telangana', 'Rajasthan',
       'Uttar Pradesh', 'Tamil Nadu', 'Gujarat', 'Bihar', 'Uttarakhand',
       'Jharkhand', 'Punjab', 'Assam', 'Chhatisgarh', 'Goa', 'Kerala',
       'Andhra Pradesh', 'Jammu & Kashmir', 'Karnataka', 'Tripura',
       'Meghalaya', 'Nagaland', 'Puducherry', 'Sikkim',
       'Arunachal Pradesh', 'Haryana', 'Manipur', 'Mizoram', 'Diu'],
      dtype=object)

In [19]:
df["Round"].max()

6

In [28]:
df["Quota"].unique()

array(['AI', 'HS', 'OS'], dtype=object)

In [ ]:
# re rank based on round 1 ke open category ka closing ranks!

In [27]:
df[df["Institute"] == "National Institute of Technology Karnataka, Surathkal"]["Quota"].unique()

array(['HS', 'OS'], dtype=object)

In [32]:
neet_report = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Report.csv")
neet_stud = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Table 1.csv", header=1)

neet_stud.columns, neet_report.columns

(Index(['S. No.', 'Rollno', 'Name', 'Quota', 'AIR', 'Category', 'optionN o',
        'Institute Code', 'Institute Name', 'Subject', 'Allotted Category',
        'Allotte d ph', 'Admitte dRound'],
       dtype='object'),
 Index(['Institute Name', 'State', 'Subject', 'General', 'EW', 'SC', 'ST',
        'OBC', 'General_PWD', 'EW_PWD', 'SC_PWD', 'ST_PWD', 'OBC_PWD',
        'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
        'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
        'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
       dtype='object'))

In [186]:
from collections import defaultdict

institute_code_to_names = defaultdict(list)
institute_name_to_codes = {}
for it, row in neet_stud.iterrows():
    code, name = row["Institute Code"], row["Institute Name"]

    if name not in institute_code_to_names[code]:
        institute_code_to_names[code].append(name)

    institute_name_to_codes[name] = code

In [ ]:
for k, v in institute_code_to_names.items():
    if len(v) > 1:
        print(k, v)
        break

In [167]:
# i am picking the first one that appears
code_to_college_mapper = {}
for k, v in institute_code_to_names.items():
    code_to_college_mapper[k] = v[0]

assert len(institute_code_to_names) == len(code_to_college_mapper)

In [168]:
# for each round, for each subject, for each quota, 
# using ph as pwd -- physically handicapped
# consider allotted cat or actual cat?
# what to do about quota? Employee State Insurance, etc.
# all of this should be for each institute na ... institute code!!
deter_cols = ["Quota", "Admitte dRound", "Allotte d ph", "Subject"]

output_cols = ["Institute Code", "Institute Name"]

close_mapper = defaultdict(lambda : 0)
open_mapper = defaultdict(lambda : float("inf"))

for it, row in neet_stud.iterrows():

    deter_col_vals = [row[x] for x in deter_cols]
    
    cat = row["Allotted Category"]
    if cat == "OP":
        cat = "General"
    
    deter_col_vals.append(cat)

    inst_code = row["Institute Code"]
    output_col_vals = [inst_code, code_to_college_mapper[inst_code]]

    all_cols = deter_col_vals + output_col_vals

    all_cols = tuple(all_cols) # hashable

    if close_mapper[all_cols] < row["AIR"]:
        close_mapper[all_cols] = row["AIR"]

    if open_mapper[all_cols] > row["AIR"]:
        open_mapper[all_cols] = row["AIR"]




In [169]:
for k, v in close_mapper.items():
    print(k, v)
    break

('Open Seat Quota', 1, 'NO', 'MBBS', 'General', 200502, 'AIIMS, New Delhi,AIIMS ANSARI NAGAR EAST AUROBINDO MARG NEW DELHI 110029, Delhi (NCT), 110029') 55.0


In [170]:
cat_unique_values = ["SC", "ST", "General", "OBC", "EW"]

In [171]:
all_dic = []
selectors = [0, 1, 2, 3, 4, 5, 6]
selector_names = ["Quota", "Round", "is_PWD", "Subject", "Seat Type", "Institute Code", "Institute Name"]

for k, v in close_mapper.items():
    dic = {}
    for key_index, name in zip(selectors, selector_names):
        dic[name] = k[key_index]

    if k in open_mapper:
        dic["Opening Rank"] = open_mapper[k]

    dic["Closing_Rank"] = v
    
    all_dic.append(dic)

    # break

all_dic = pd.DataFrame(all_dic)

In [172]:
# time to rank institutes!!
# find round 1 ka opening rank.. rank based on it!

code_to_open_ranks = defaultdict(lambda : float("inf"))

for it, row in all_dic.iterrows():
    # print(row["Round"] == 1 , row["Category"] == "General" , row["is_PWD"] == "NO", row["Quota"] == "Open Seat Quota")
    if code_to_open_ranks[row["Institute Code"]] > row["Opening Rank"]:
        code_to_open_ranks[row["Institute Code"]] = row["Opening Rank"]

        # break

In [173]:
# now sort the dict based on this
# sort keys of dict based on values
ranking = sorted(code_to_open_ranks.items(), key = lambda item: item[1])

code_to_rank_mapper = {}

rank_idx = 1
for k, v in ranking:
    code_to_rank_mapper[k] = rank_idx
    rank_idx += 1

In [174]:
len(all_dic["Institute Code"].unique()), len(code_to_open_ranks)

(483, 483)

In [175]:
all_dic["College Rank"] = all_dic["Institute Code"].map(code_to_rank_mapper)

In [177]:
# combine pwd and category
all_dic["Seat Type"] = all_dic.apply(lambda row: row["Seat Type"] + " (PwD)" if row["is_PWD"] != "NO" else row["Seat Type"], axis = 1)

In [151]:
all_dic.columns, df.columns

(Index(['Quota', 'Round', 'is_PWD', 'Subject', 'Category', 'Institute Code',
        'Institute Name', 'Opening Rank', 'Closing_Rank', 'College Rank'],
       dtype='object'),
 Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
        'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
        'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
        'Category Match', '500', 'OBC-NCL'],
       dtype='object'))

In [183]:
all_dic = all_dic.rename(columns={
    "Category": "Seat Type",
    "Institute Name": "Institute",
    "Subject": "Academic Program Name",
    "Closing_Rank": "Closing Rank"
}) 

In [179]:
def replace_cat(row):
    cat = row["Seat Type"]
    if cat == "General":
        cat = "OPEN"
    elif cat == "General (PwD)":
        cat = "OPEN (PwD)"
    elif cat == "EW":
        cat = "EWS"
    elif cat == "EW (PwD)":
        cat = "EWS (PwD)"
    elif cat == "OBC":
        cat = "OBC-NCL"
    elif cat == "OBC (PwD)":
        cat = "OBC-NCL (PwD)"

    return cat

all_dic["Seat Type"] = all_dic.apply(lambda row: replace_cat(row), axis = 1)



In [187]:
# figuring out state for each..
# merge on institute name
print(neet_stud.shape, neet_report.shape)
inst_code_to_state_map = {}
for it, row in pd.merge(neet_stud, neet_report, on=["Institute Name"]).iterrows():
    name, state = row["Institute Name"], row["State"]

    inst_code_to_state_map[institute_name_to_codes[name]] = state



all_dic["State"] = all_dic["Institute Code"].map(inst_code_to_state_map)


(23059, 13) (433, 25)


In [188]:
for cat in all_dic["Seat Type"].unique():
    all_dic_category = all_dic[all_dic["Seat Type"] == cat]

    all_dic_category.to_json(f"public/data/NEET/{cat}.json", orient="records")

In [190]:
all_dic.to_csv("Surya - NEET Data.csv")

In [78]:
schol_data = pd.read_csv("scholarship_data.csv")

In [79]:
schol_data.columns

Index(['Scholarship Name', 'No. of awards', 'Scholarship Amount',
       'Scholarship Frequency', 'Status', 'Class 10 or below can apply',
       'Class 11 can Apply', 'Class 12 can Apply', '12th Passed Can Apply',
       '2nd/3rd Yr Eligible', 'Diploma/ITI', 'Eligible for PG', 'Gender',
       'Family Income (in INR)', 'State', 'City', 'Last Date', 'Stream',
       'Eligibility', 'Benefits', 'Doc Required', 'Application Link',
       'Pre-filled Form Link'],
      dtype='object')

In [23]:
# def expand_rows(df, columns):
#     df_expanded = df.copy()
#     for column in columns:
#         df_expanded = df_expanded.drop(column, axis=1).join(
#             df_expanded[column].dropna().str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename(column)
#         )
#     return df_expanded

# # Expanding rows for 'State' and 'City' columns
# expanded_df = expand_rows(schol_data, ['State', 'City ', 'Stream', 'Gender'])

In [80]:
# processing grade columns
grade_columns = [
    "Class 10 or below can apply",
    "Class 11 can Apply",
    "Class 12 can Apply",
    "12th Passed Can Apply",
    "2nd/3rd Yr Eligible",
    "Diploma/ITI",
    "Eligible for PG"
]

grade_form = """Currently in 10th or below
Currently in 11th
Currently in 12th
12th pass/ Currently in UG 1st year
Currently in UG 2nd/3rd/4th year
Currently in PG
Currently in Diploma
Any""".splitlines()

grade_labels = ["10", "11", "12", "12_pass", "UG", "Diploma", "PG"]

def create_grade_array(row):
    grades = []
    for col, label in zip(grade_columns, grade_labels):
        if row[col] == "Yes":  # if the value is truthy
            grades.append(label)
    return grades

schol_data['Grade'] = schol_data.apply(create_grade_array, axis=1)


In [81]:
# family income processing
import re
def extract_income(value):
    if pd.isnull(value): return value
    match = re.search(r'(\d+(\.\d+)?)', value)  # matches the first numeric value (integer or decimal)
    if match:
        return float(match.group(1))  # return as float to handle decimals
    return value  # return original value if no match

# Apply the function to the Salary column
schol_data['Family Income (in INR)'] = schol_data['Family Income (in INR)'].apply(extract_income)

In [82]:
schol_data["Family Income (in INR)"]

0      NaN
1      8.0
2      6.0
3      8.0
4      6.0
      ... 
177    NaN
178    6.0
179    6.0
180    NaN
181    6.0
Name: Family Income (in INR), Length: 182, dtype: float64

In [28]:

# expanded_df = expanded_df.rename(columns={"City ": "City"})

In [25]:
# expanded_df["State"].unique()

array([nan, 'Goa', 'Gujarat', 'Maharashtra', 'Himachal Pradesh',
       'Andhra Pradesh', 'Karnataka', 'Telangana', 'Tamil Nadu', 'Kerala',
       'Bihar', 'Haryana', 'Uttarakhand', 'West Bengal', 'Assam', 'Delhi',
       'Rajasthan', 'Jharkhand', 'Punjab', 'Odisha', 'Chhattisgarh',
       'Madhya Pradesh', 'Uttar Pradesh'], dtype=object)

In [26]:
# expanded_df["City"].unique()

array([nan, 'Mumbai', 'Delhi NCR', 'Ahmedabad', 'Bangalore', 'Pune',
       'Chennai', 'Nagpur', 'Hyderabad'], dtype=object)

In [51]:
# expanded_df.iloc[92]

In [83]:
schol_data.to_json("public/data/scholarships/scholarship_data.json", orient="records")

In [70]:
schol_data.head()

,Scholarship Name,No. of awards,Scholarship Amount,Scholarship Frequency,Status,Class 10 or below can apply,Class 11 can Apply,Class 12 can Apply,12th Passed Can Apply,2nd/3rd Yr Eligible,...,State,City,Last Date,Stream,Eligibility,Benefits,Doc Required,Application Link,Pre-filled Form Link,Grade
0,Samsung Star,NaN,NaN,One-time,Closed,No,No,No,Yes,No,...,NaN,NaN,NaN,Engineering (BTech),Be enrolled in B.Tech/B.Tech+M.Tech dual degre...,The selected students will receive a scholarsh...,Self-attested photograph\nSelf-certified Class...,https://www.samsung.com/in/microsite/sapne-hue...,https://docs.google.com/forms/d/e/1FAIpQLSecfF...,[12_pass]
1,Colgate - BDS,NaN,"75,000",One-time,Open,No,No,No,Yes,Yes,...,"Goa, Gujarat, Maharashtra, Himachal Pradesh, A...",NaN,10/31/2024,Medical (BDS),Open for students who are currently enrolled i...,"₹75,000",Photo identity proof (Aadhaar Card)\nPassport-...,https://www.buddy4study.com/page/colgate-keep-...,https://docs.google.com/forms/d/e/1FAIpQLSecfF...,"[12_pass, UG]"
2,Colgate - Class XI,NaN,"20,000",2 years,Closed,No,Yes,No,No,No,...,NaN,NaN,3/31/2023,NaN,Must have passed Class 10 board examination in...,"INR 20,000 per annum for 2 years",Passport-sized photograph\nValid ID proof – Ei...,https://www.colgate.com/en-in/smile-karo-aur-s...,https://docs.google.com/forms/d/e/1FAIpQLSecfF...,[11]
3,Colgate Keep India Smiling Scholarship for MDS...,NaN,"75,000",One-time,Open,No,No,No,No,No,...,"Goa, Gujarat, Maharashtra, Himachal Pradesh, A...",NaN,10/31/2024,Medical (BDS),Open for students currently enrolled in the MD...,"One-time scholarship of INR 75,000",Photo identity proof (Aadhaar Card)\nPassport-...,NaN,https://docs.google.com/forms/d/e/1FAIpQLSecfF...,[PG]
4,Colgate - Graduate,NaN,"30,000",3 years,Closed,No,No,No,Yes,No,...,NaN,NaN,3/31/2023,"Science (BSc), Commerce (BCom), Arts (BA)",Must have passed Class 12 board examination in...,"INR 30,000 per year for 3 years",Passport-sized photograph\nValid ID proof – Ei...,https://www.colgate.com/en-in/smile-karo-aur-s...,https://docs.google.com/forms/d/e/1FAIpQLSecfF...,[12_pass]


In [77]:
# Defining the filter based on the URL parameters
df = schol_data
status_filter = 'Closed'
grade_filter = '12_pass'
stream_filter = 'Engineering (BTech)'
gender_filter = 'Girls'
familyIncome_filter = 2
state_filter = 'All India'

# Applying the filters to the DataFrame
filtered_df = df[
    (df['Status'] == status_filter) &
    (df['Grade'].apply(lambda grades: grade_filter in grades)) &
    (df['Stream'] == stream_filter) &
    # (df['Gender'] == gender_filter) &
    # (df['Family Income (in INR)'] <= familyIncome_filter) &  # Assuming "familyIncome=2" means <= 2
    ((df['State'] == state_filter) | (state_filter == 'All India'))  # 'All India' as a wildcard
]

# Display the filtered DataFrame
print(filtered_df)

                                      Scholarship Name  No. of awards  \
0                                         Samsung Star            NaN   
11                Rolls Royce Unnati Women Engineering            NaN   
13   Schaeffler India Hope Engineering Scholarship ...            NaN   
16               Deutsche Bank Engineering Scholarship            NaN   
18   Ericsson Empowering Girls Scholarship for Engi...            NaN   
26          Panasonic Ratti Chhatr Scholarship 2022-23            NaN   
63    Medhaavi Engineering Scholarship Program 2023-24            NaN   
69                        Siemens Scholarship\nProgram            NaN   
81                  Aspire Scholarship Program 2023-24           28.0   
93   Alstom India Scholarship for STEM Education at...          115.0   
96   Alstom India Scholarship for STEM Education at...           44.0   
98   Infocepts Innovate for Impact Scholarship 2023-24            NaN   
151                   IET India Scholarship Award 2

####################################

In [46]:
mhcet_data = pd.read_csv("MHT CET 2023 All Cutoffs - Cutoffs.csv")
mhcet_cat = pd.read_csv("MHT CET 2023 All Cutoffs - Categories.csv")

In [47]:
mhcet_cat = mhcet_cat.rename(columns = {"Unnamed: 0": "Cat"})

In [48]:
for col in mhcet_cat.columns:
    print(mhcet_cat[col].unique())

['GOPENS' 'GOPENH' 'GSCS' 'GSTS' 'GVJS' 'GNT1S' 'GNT2S' 'GNT3S' 'GOBCS'
 'LOPENS' 'LSCS' 'LSTS' 'LVJS' 'LNT1S' 'LNT2S' 'LNT3S' 'LOBCS' 'PWDOPENS'
 'DEFOPENS' 'TFWS' 'EWS' 'PWDROBC\nS' 'DEFROBCS' 'Unnamed: 1' 'PWDRSCS'
 'GSCH' 'GVJH' 'GNT1H' 'GOBCH' 'LOPENH' 'LOBCH' 'GOPENO' 'GSCO' 'GNT3O'
 'GOBCO' 'LOPENO' 'LSCH' 'GSTH' 'LNT2H' 'GNT2H' 'GNT3H' 'LNT1H' 'LSTH'
 'GNT2O' 'LSTO' 'LOBCO' 'LNT3H' 'PWDOPENH' 'GSTO' 'GVJO' 'LSCO' 'LNT2O'
 'LVJO' 'LNT3O' 'LVJH' 'LNT1O' 'GNT1O' 'PWDROBC' 'PWDOBCS' 'DEFOBCS'
 'DEFRSCS' 'ORPHAN' 'MI' 'DEFRNT3S' 'DEFRNT2S' 'PWDRNT2S' 'DEFRNT1S'
 'DEFRVJS' 'PWDSCS' 'DEFSCS' 'PWDRNT3S' 'PWDRSCH' 'PWDRNT1S' 'PWDRSTS'
 'PWDOBCH' 'PWDRNT2H' 'PWDSCH' 'PWDRNT1H' 'PWDROBC\nH' 'Unnamed: 2'
 'PWDRSTH' 'PWDRVJH' 'PWDRVJS']
['General' 'Female']
['Open' 'SC' 'ST' 'VJ' 'NT' 'OBC' nan 'EWS' 'Orphan' 'Religious Minority']
['Maharashtra' 'Other']
['No' 'Yes']
['No' 'Yes']
['MHT CET']


In [49]:
# required columns
# Gender, Category, State, PWD, Defense, Closing Rank, Institute, Course

all_dic = []
for it, row in mhcet_data.iterrows():
    for k, v in row.items():
        
        if k in ["College", "Course", "Unnamed: 1", "Unnamed: 2"]:
            continue
        else:
            dic = {}
            dic["Institute"] = row["College"]
            dic["Course"] = row["Course"]
            cat_row = dict(mhcet_cat[mhcet_cat["Cat"] == k].iloc[0])
            
            dic["Category"] = cat_row["Category"]
            if cat_row["Gender"] == "General":
                dic["Gender"] = "Gender-Neutral"
            elif cat_row["Gender"] == "Female":
                dic["Gender"] = "Female-Only"
            dic["Defense"] = cat_row["Defense Ward"]
            dic["PWD"]= cat_row["PWD"]
            dic["State"] = cat_row["State"]
            dic["Category_Key"] = k

            if v == "" or pd.isna(v): continue # no rank available

            v = str(v)

            if "\n" in v:
                v = v.split("\n")[0]
            elif " " in v:
                v = v.split(" ")[0]
            elif "." in v:
                v = v.split(".")[0]

            dic["Closing Rank"] = v

            all_dic.append(dic)
            # break 
    
    # break 
            # dic["Category"] = k

            




In [50]:
final_mhcet_data = pd.DataFrame(all_dic)

In [51]:
final_mhcet_data.to_json("public/data/MHTCET/mhtcet_data.json", orient="records")

In [52]:
for it, row in final_mhcet_data.iterrows():
    # Show Both Maharashtra No No General
    # print(row)
    if row["Category"] == "Open" and row["Gender"] == "Gender-Neutral" and row["State"] == "Maharashtra" and row["PWD"] == "No" and row["Defense"] == "No":
        print(row)
    break

Institute       Government College of Engineering, Amravati
Course                                    Civil Engineering
Category                                               Open
Gender                                       Gender-Neutral
Defense                                                  No
PWD                                                      No
State                                           Maharashtra
Category_Key                                         GOPENS
Closing Rank                                          45820
Name: 0, dtype: object


In [50]:
final_mhcet_data.columns

Index(['Institute', 'Course', 'Category', 'Gender', 'Defense', 'PWD', 'State',
       'Category_Key', 'Closing Rank'],
      dtype='object')

In [53]:
for col in final_mhcet_data.columns:
    if col not in ["Institute", "Closing Rank", "Course"]:
        print(final_mhcet_data[col].unique())

['Open' 'SC' 'ST' 'VJ' 'NT' 'OBC' nan 'EWS' 'Orphan' 'Religious Minority']
['General' 'Female']
['No' 'Yes']
['No' 'Yes']
['Maharashtra']
['GOPENS' 'GSCS' 'GSTS' 'GVJS' 'GNT1S' 'GNT2S' 'GNT3S' 'GOBCS' 'LOPENS'
 'LSCS' 'LSTS' 'LVJS' 'LNT1S' 'LNT2S' 'LNT3S' 'LOBCS' 'PWDOPENS'
 'DEFOPENS' 'TFWS' 'EWS' 'PWDROBC\nS' 'DEFROBCS' 'PWDRSCS' 'GOPENH' 'GSCH'
 'GVJH' 'GNT1H' 'GOBCH' 'LOPENH' 'LOBCH' 'LSCH' 'GNT2H' 'GNT3H' 'LNT1H'
 'GSTH' 'LSTH' 'LNT3H' 'PWDOPENH' 'LNT2H' 'LVJH' 'PWDROBC' 'PWDOBCS'
 'DEFOBCS' 'DEFRSCS' 'DEFRNT3S' 'ORPHAN' 'DEFRNT2S' 'DEFRVJS' 'PWDRNT2S'
 'MI' 'PWDSCS' 'DEFSCS' 'DEFRNT1S' 'PWDRNT1S' 'PWDRSTS' 'PWDRNT3S'
 'PWDOBCH' 'PWDSCH' 'PWDRNT1H' 'PWDROBC\nH' 'PWDRSTH']


In [58]:
final_mhcet_data[final_mhcet_data["Category_Key"] == "LOBCO"]

,Institute,Course,Category,Gender,Defense,PWD,State,Category_Key,Closing Rank


In [66]:
kcet_data = pd.read_csv("kcet_counselling_data_2022.csv")

print(kcet_data.shape)
print(kcet_data.columns)

(138775, 14)
Index(['ENTRY No.', 'OFFSET', 'Course Type', 'category', 'filename',
       'sheetname', 'college', 'course', 'Caste', 'Category', 'Language',
       'Rural / Urban', 'Label', 'Closing Rank'],
      dtype='object')


In [70]:
all_dic = []
for it, row in kcet_data.iterrows():
    dic = {}
    if pd.isna(row["college"]): continue
    if pd.isna(row["Course Type"]): continue
    dic["Institute"] = row["college"].replace("\xa0", " ")
    dic["Course Type"] = row["Course Type"]
    dic["Course"] = row["course"]
    dic["Category"] = row["Caste"]
    dic["State"] = row["Category"]
    dic["Language"] = row["Language"]
    dic["Rural/Urban"] = row["Rural / Urban"]
    dic["Category_Key"] = row["Label"]

    v = row["Closing Rank"]
    if v == "" or pd.isna(v): continue # no rank available

    v = str(v)

    if "\n" in v:
        v = v.split("\n")[0]
    elif " " in v:
        v = v.split(" ")[0]
    elif "." in v:
        v = v.split(".")[0]

    dic["Closing Rank"] = v

    all_dic.append(dic)

In [71]:
final_kcet_data = pd.DataFrame(all_dic)

In [72]:
for col in final_kcet_data.columns:
    print(col)
    print(final_kcet_data[col].unique())

Institute
['M001  Bangalore Medical College, Bangalore'
 'M003  Kempegowda Institute of Medical Sciences, Bangalore'
 'M004  M.S.Ramaiah Medical College, Bangalore'
 'M005  Khaja Bande Navaz Institute Of Medical Sciences, Gulbarga'
 'M008  M.V.J.Medical College and Research Hospital, Channasandra, Bangalore'
 'M009  Vydehi Institute of Medical Science and Research Centre, Bangalore'
 'M010  A.J.Institute of Medical Sciences, Mangalore'
 'M020  Sri Basaveshwara Medical College and Hospital, Chitradurga'
 'M021  Mysore Medical College, Mysore'
 'M023  Adichunchanagiri Institute of Medical Sciences, Bellur, Mandya Dist.'
 'M024  Mandya Institute of Medical Sciences, Mandya'
 'M025  Hassan Institute of Medical Sciences, Hassan'
 'M031  Karnataka Institute of Medical Sciences, Hubli'
 'M035  S. Nijalingappa Medical College and Research Centre, Bagalkot'
 'M036  SDM College of Medical Sciences and Hospital, Dharwad'
 'M037  Belgaum Institute of Medical Sciences, Belgaum'
 'M042  Mahadevappa 

In [73]:
final_kcet_data.columns

Index(['Institute', 'Course Type', 'Course', 'Category', 'State', 'Language',
       'Rural/Urban', 'Category_Key', 'Closing Rank'],
      dtype='object')

In [74]:
final_kcet_data.to_json("public/data/KCET/kcet_data.json", orient="records")

In [34]:
final_kcet_data[final_kcet_data["Language"] == "Kannada"]["State"].unique()

array(['Karnataka'], dtype=object)

In [54]:
jac_data = pd.read_csv("JAC 2023 Cutoffs - Cutoffs.csv")

jac_data.head()

,Institute,Course,EWCWD,EWCWO,EWGLD,EWGLO,EWGND,EWGNO,EWPDD,EWPDO,...,SCPDO,SG,STCWD,STCWO,STGLD,STGLO,STGND,STGNO,STPDD,STPDO
0,Delhi Technological University (DTU),Computer Science and Engineering (CSE),NaN,NaN,NaN,NaN,24129,7658.0,NaN,NaN,...,NaN,14322.0,NaN,NaN,NaN,NaN,296967.0,96383.0,NaN,NaN
1,Delhi Technological University (DTU),Information Technology (IT),NaN,NaN,NaN,NaN,29278,8189.0,NaN,NaN,...,NaN,22172.0,NaN,NaN,NaN,NaN,412802.0,121909.0,NaN,NaN
2,Delhi Technological University (DTU),Software Engineering (SE),NaN,NaN,NaN,NaN,33772,9246.0,NaN,NaN,...,NaN,23161.0,NaN,NaN,NaN,NaN,500239.0,116361.0,NaN,NaN
3,Delhi Technological University (DTU),Mathematics and Computing (MCE),NaN,NaN,NaN,NaN,37809,9555.0,NaN,NaN,...,NaN,25761.0,NaN,NaN,NaN,NaN,626554.0,143319.0,NaN,NaN
4,Delhi Technological University (DTU),Electronics and Communication Engineering (ECE),NaN,NaN,NaN,NaN,46787,13415.0,NaN,NaN,...,NaN,33648.0,NaN,NaN,NaN,NaN,917669.0,168274.0,NaN,NaN


In [55]:
jac_cat = pd.read_csv("JAC 2023 Cutoffs - Category_Trans.csv")

jac_cat.head()

,Category,Caste,Region,Gender,Defense Ward,Physically Disabled
0,GNCWD,General,Delhi,Any,Yes,No
1,GNPDD,General,Delhi,Any,No,Yes
2,GNGLD,General,Delhi,Female,No,No
3,GNGND,General,Delhi,Any,No,No
4,GNCWO,General,Outside Delhi,Any,Yes,No


In [62]:
all_dic = []
for it, row in jac_data.iterrows():
    for k, v in row.items():
        
        if k in ["Institute", "Course"]:
            continue
        else:
            if v == "" or pd.isna(v): continue # no rank available
            
            dic = {}
            dic["Institute"] = row["Institute"]
            dic["Course"] = row["Course"]
            cat_row = dict(jac_cat[jac_cat["Category"] == k].iloc[0])
            
            dic["Category"] = cat_row["Caste"]
            if cat_row["Gender"] == "Any":
                dic["Gender"] = "Gender-Neutral"
            elif cat_row["Gender"] == "Female":
                dic["Gender"] = "Female-Only"
            dic["Defense"] = cat_row["Defense Ward"]
            dic["PWD"]= cat_row["Physically Disabled"]
            dic["State"] = cat_row["Region"]
            dic["Category_Key"] = k

            v = str(v)

            if "\n" in v:
                v = v.split("\n")[0]
            elif " " in v:
                v = v.split(" ")[0]
            elif "." in v:
                v = v.split(".")[0]

            dic["Closing Rank"] = v

            all_dic.append(dic)




In [63]:
jac_final = pd.DataFrame(all_dic)

In [64]:
jac_final.head()

,Institute,Course,Category,Gender,Defense,PWD,State,Category_Key,Closing Rank
0,Delhi Technological University (DTU),Computer Science and Engineering (CSE),EWS,Gender-Neutral,No,No,Delhi,EWGND,24129
1,Delhi Technological University (DTU),Computer Science and Engineering (CSE),EWS,Gender-Neutral,No,No,Outside Delhi,EWGNO,7658
2,Delhi Technological University (DTU),Computer Science and Engineering (CSE),General,Gender-Neutral,No,No,Delhi,GNGND,12948
3,Delhi Technological University (DTU),Computer Science and Engineering (CSE),General,Gender-Neutral,No,No,Outside Delhi,GNGNO,4876
4,Delhi Technological University (DTU),Computer Science and Engineering (CSE),OBC,Gender-Neutral,No,No,Delhi,OBGND,49995


In [10]:
for col in jac_final.columns:
    print(jac_final[col].unique())

['Delhi Technological University (DTU)'
 'Netaji Subhas University of Technology (NSUT)'
 'Indraprastha Institute of Information Technology Delhi (IIIT Delhi)'
 'Indira Gandhi Delhi Technical University for Women (IGDTUW)']
['Computer Science and Engineering (CSE)' 'Information Technology (IT)'
 'Software Engineering (SE)' 'Mathematics and Computing (MCE)'
 'Electronics and Communication Engineering (ECE)'
 'Electrical Engineering (EE)' 'Mechanical Engineering (ME)'
 'Automotive Engineering (MAM)' 'Engineering Physics (EP)'
 'Chemical Engineering (CHE)' 'Civil Engineering (CE)'
 'Production & Industrial Engineering (PIE)'
 'Environmental Engineering (ENE)' 'Bio-Technology (BT)'
 'Computer Science and Engineering (Artificial Intelligence) (CSAI)'
 'Computer Science and Engineering (Data Science) (CSDS)'
 'Information Technology (Network and Information Security) (ITNS)'
 'Mathematics and Computing (MAC)'
 'Electronics and Communication Engineering (Internet of Things) (EIOT)'
 'Instrume

In [65]:
jac_final.to_json("public/data/JEE/jac_data.json", orient="records")